In [1]:
# !pip install numpy==1.15
!pip  install --upgrade tables

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [2]:




# Upload the train file from your local drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd /content/drive/My\ Drive/Over_Sampling

/content/drive/My Drive/Over_Sampling


In [0]:
from sklearn.datasets import make_multilabel_classification

import torch
import os
import pandas as pd
import numpy as np
from collections import defaultdict
import random

In [0]:
y = np.array([[1,2,3],[2,1,3,4],[4,5,6,1],[1,5,3]])

In [0]:
from ML_ROS import IRLbl, MeanIR

In [7]:
IRLbl(y)

1 4
2 2
3 3
4 2
5 2
6 1


defaultdict(int,
            {1: 1.0, 2: 2.0, 3: 1.3333333333333333, 4: 2.0, 5: 2.0, 6: 4.0})

In [0]:
def SCUMBLE(y):
    ratios = IRLbl(y)
    
    scumbles = []
    for sample in y:
        am = 0
        gm = 1
        for label in sample:
            ratio = ratios[label]
            am += ratio
            gm *= ratio
        
        n_labels  = len(sample)
        am/=n_labels
        gm = np.power(gm, 1/n_labels)
        
        scumble = 1-(gm/am)
        scumbles.append(scumble)
        

    return np.mean(scumbles), scumbles

In [18]:
# mean, scumbles = SCUMBLE(y)
mean, scumbles = SCUMBLE(y)

mean

AMOXICILLIN ORAL SUPENSION 125mg/30ml 50
PCM ORAL SUSPENSION 250mg 24
RANITIDINE 150 mg 1710
AMOXICILLIN 500mg 405
AMOXICILLIN 250mg 219
AMOXICILLIN 500mg + POTASSIUM CLAVULANATE 5G/30ML 107
AMLODIPINE BISILATE 5mg 1219
CEFIXIME 100mg 76
PCM ORAL SUSPENSION 125mg 30
PCM ORAL SUSPENSION 120mg 18
ATENOLOL 50mg 370
PANTOPRAZOLE 40 mg 3240
PARACETAMOL 500mg 3191
DICLOFENAC Ointment 2998
VITAMIN B Complex +C 931
VITAMIN B Complex 1957
METFORMIN 500mg 1194
LOSARTAN 50mg 237
ATENOLOL 25mg 170
METFORMIN 500mg - SR 114
GLIMEPERIDE 2 mg + METFORMIN 500mg 462
FERROUS ASCORBATE 100mg + FOLIC ACID 1.5mg 594
METRONIDAZOLE 400mg 176
CLOTRIMAZOLE Powder 10
FLUCANAZOLE 150mg 175
CETIRIZINE Di HCl 10mg 1606
AZITHROMYCIN 500mg 222
ACECLOFENAC 100mg 898
CALCIUM CARBONATE 1250mg & VIT D3 250 IU 3868
CEFIXIME 200mg 449
ITRACONAZOLE 100 mg 65
VITAMIN E 400 mg 106
HEMATINIC SYRUP - Protein, Iron, Folic Acid and Vitamin B 12 (pediatric) 292
DERIPHYLLINE 100 mg 263
Povidone-Iodine IP 5% 32
AMOXICILLIN 500mg + P

0.2231913082230534

In [0]:
def REMEDIAL(X,y):
    from copy import deepcopy
    scumble, scumbles = SCUMBLE(y)
    mean_ir, ratios =  MeanIR(y, True)
    updated_X = []
    updated_y = []
    for i,_ in enumerate(zip(X,y)):
        X_i = X[i]
        y_i = y[i]
        if scumbles[i] > 0:
            c_X_i = deepcopy(X_i)
            c_y_i = deepcopy(y_i)
            c_y_i = [label for label in c_y_i if ratios[label] > 0.05*mean_ir]   # retain minority labels
            y_i = [label for label in y_i if ratios[label] <= 0.1*mean_ir]      # retain majority labels
            print(f'{y_i} {c_y_i}')
            if c_y_i != []:
                updated_X.append(c_X_i)                                       # add minority samples
                updated_y.append(c_y_i)                                       # to the updated lists

        if y_i != []:
            updated_X.append(X_i)                                             # add actual or majority samples
            updated_y.append(y_i)                                             # to the updated lists
        
    return updated_X, updated_y

In [0]:
X,y = make_multilabel_classification(n_classes=10,n_samples=100,allow_unlabeled=False,)
y = np.where(y==1)

y

y = pd.DataFrame({'row':y[0], 'col':y[1]})
y

y = y.groupby('row')['col'].apply(list)

list(y)
len(X),len(y)


(100, 100)

In [14]:
X_,y_ = REMEDIAL(X,y)

Streaming output truncated to the last 5000 lines.
[] ['NEOMYCIN 7.5gm Ointment', 'CEFIXIME 100mg']
['CETIRIZINE Di HCl 10mg', 'AZITHROMYCIN 500mg', 'CIPROFLOXACIN Eye/Ear Drops 5ml', 'RANITIDINE 150 mg'] ['AZITHROMYCIN 500mg', 'CIPROFLOXACIN Eye/Ear Drops 5ml']
['GLIMEPIRIDE  2mg', 'METFORMIN 500mg', 'MULTIVIT&MINERALS+METHYLCOBLAMIN+ANTIOXIDANTS', 'PANTOPRAZOLE 40 mg', 'PARACETAMOL 500mg'] []
['AMOXICILLIN 500mg + POTASSIUM CLAVULANIC ACID 125mg', 'PANTOPRAZOLE 40 mg', 'DOMPERIDONE 10mg'] ['DOMPERIDONE 10mg']
['DICLOFENAC POTASSIUM 50mg', 'PARACETAMOL 500mg', 'MULTIVIT&MINERALS+METHYLCOBLAMIN+ANTIOXIDANTS'] []
[] ['NEOMYCIN 7.5gm Ointment', 'CEFIXIME 100mg']
['GABAPENTIN 300mg + METHYLCOBALAMIN 500mg', 'PANTOPRAZOLE 40 mg', 'CALCIUM CARBONATE 1250mg & VIT D3 250 IU', 'DERIPHYLLINE 100 mg', 'AMOXICILLIN 500mg', 'DERIPHYLLINE RETARD 150mg', 'PARACETAMOL 250mg', 'VITAMIN B Complex +C'] ['PARACETAMOL 250mg']
['CALCIUM CARBONATE 1250mg & VIT D3 250 IU', 'VITAMIN B Complex +C'] []
['PARACE

In [0]:
data = pd.read_hdf('../UC_datasets/pure_datasets/data.hdf')

In [0]:
X = data.symptoms
y = data.label

In [19]:
# SCUMBLE(y), 
mean, scumbles = SCUMBLE(y_)

mean# SCUMBLE(y_)

AMOXICILLIN ORAL SUPENSION 125mg/30ml 50
PCM ORAL SUSPENSION 250mg 24
RANITIDINE 150 mg 1710
AMOXICILLIN 500mg 405
AMOXICILLIN 250mg 419
AMOXICILLIN 500mg + POTASSIUM CLAVULANATE 5G/30ML 107
CEFIXIME 100mg 76
AMLODIPINE BISILATE 5mg 1219
PCM ORAL SUSPENSION 125mg 30
PCM ORAL SUSPENSION 120mg 18
ATENOLOL 50mg 370
PANTOPRAZOLE 40 mg 3240
PARACETAMOL 500mg 3191
DICLOFENAC Ointment 2998
VITAMIN B Complex +C 931
VITAMIN B Complex 1957
ATENOLOL 25mg 332
METFORMIN 500mg 1194
LOSARTAN 50mg 237
METFORMIN 500mg - SR 114
GLIMEPERIDE 2 mg + METFORMIN 500mg 462
FERROUS ASCORBATE 100mg + FOLIC ACID 1.5mg 594
METRONIDAZOLE 400mg 337
CLOTRIMAZOLE Powder 10
FLUCANAZOLE 150mg 337
CETIRIZINE Di HCl 10mg 1606
AZITHROMYCIN 500mg 434
ACECLOFENAC 100mg 898
CALCIUM CARBONATE 1250mg & VIT D3 250 IU 3868
ITRACONAZOLE 100 mg 65
CEFIXIME 200mg 449
VITAMIN E 400 mg 106
HEMATINIC SYRUP - Protein, Iron, Folic Acid and Vitamin B 12 (pediatric) 292
DERIPHYLLINE 100 mg 263
Povidone-Iodine IP 5% 32
AMOXICILLIN 500mg + P

0.12443896964456251

In [0]:
dataset = REMEDIAL(X,y)




Streaming output truncated to the last 5000 lines.
[] ['NEOMYCIN 7.5gm Ointment', 'CEFIXIME 100mg']
['CETIRIZINE Di HCl 10mg', 'AZITHROMYCIN 500mg', 'CIPROFLOXACIN Eye/Ear Drops 5ml', 'RANITIDINE 150 mg'] ['AZITHROMYCIN 500mg', 'CIPROFLOXACIN Eye/Ear Drops 5ml']
['GLIMEPIRIDE  2mg', 'METFORMIN 500mg', 'MULTIVIT&MINERALS+METHYLCOBLAMIN+ANTIOXIDANTS', 'PANTOPRAZOLE 40 mg', 'PARACETAMOL 500mg'] []
['AMOXICILLIN 500mg + POTASSIUM CLAVULANIC ACID 125mg', 'PANTOPRAZOLE 40 mg', 'DOMPERIDONE 10mg'] ['DOMPERIDONE 10mg']
['DICLOFENAC POTASSIUM 50mg', 'PARACETAMOL 500mg', 'MULTIVIT&MINERALS+METHYLCOBLAMIN+ANTIOXIDANTS'] []
[] ['NEOMYCIN 7.5gm Ointment', 'CEFIXIME 100mg']
['GABAPENTIN 300mg + METHYLCOBALAMIN 500mg', 'PANTOPRAZOLE 40 mg', 'CALCIUM CARBONATE 1250mg & VIT D3 250 IU', 'DERIPHYLLINE 100 mg', 'AMOXICILLIN 500mg', 'DERIPHYLLINE RETARD 150mg', 'PARACETAMOL 250mg', 'VITAMIN B Complex +C'] ['PARACETAMOL 250mg']
['CALCIUM CARBONATE 1250mg & VIT D3 250 IU', 'VITAMIN B Complex +C'] []
['PARACE

In [0]:
len(dataset[0])

20704

In [0]:
len(X)

16369

In [0]:
mean_ir, ratios = MeanIR(y, True)


AMOXICILLIN ORAL SUPENSION 125mg/30ml 50
PCM ORAL SUSPENSION 250mg 24
RANITIDINE 150 mg 1710
AMOXICILLIN 500mg 405
AMOXICILLIN 250mg 219
AMOXICILLIN 500mg + POTASSIUM CLAVULANATE 5G/30ML 107
AMLODIPINE BISILATE 5mg 1219
CEFIXIME 100mg 76
PCM ORAL SUSPENSION 125mg 30
PCM ORAL SUSPENSION 120mg 18
ATENOLOL 50mg 370
PANTOPRAZOLE 40 mg 3240
PARACETAMOL 500mg 3191
DICLOFENAC Ointment 2998
VITAMIN B Complex +C 931
VITAMIN B Complex 1957
METFORMIN 500mg 1194
LOSARTAN 50mg 237
ATENOLOL 25mg 170
METFORMIN 500mg - SR 114
GLIMEPERIDE 2 mg + METFORMIN 500mg 462
FERROUS ASCORBATE 100mg + FOLIC ACID 1.5mg 594
METRONIDAZOLE 400mg 176
CLOTRIMAZOLE Powder 10
FLUCANAZOLE 150mg 175
CETIRIZINE Di HCl 10mg 1606
AZITHROMYCIN 500mg 222
ACECLOFENAC 100mg 898
CALCIUM CARBONATE 1250mg & VIT D3 250 IU 3868
CEFIXIME 200mg 449
ITRACONAZOLE 100 mg 65
VITAMIN E 400 mg 106
HEMATINIC SYRUP - Protein, Iron, Folic Acid and Vitamin B 12 (pediatric) 292
DERIPHYLLINE 100 mg 263
Povidone-Iodine IP 5% 32
AMOXICILLIN 500mg + P

In [0]:

rare_med = [label  for label in ratios.keys() if ratios[label] > (0.05*mean_ir)]

In [0]:
len(rare_med)

108

In [0]:
rare_med

['AMOXICILLIN ORAL SUPENSION 125mg/30ml',
 'PCM ORAL SUSPENSION 250mg',
 'AMOXICILLIN 250mg',
 'AMOXICILLIN 500mg + POTASSIUM CLAVULANATE 5G/30ML',
 'CEFIXIME 100mg',
 'PCM ORAL SUSPENSION 125mg',
 'PCM ORAL SUSPENSION 120mg',
 'ATENOLOL 25mg',
 'METFORMIN 500mg - SR',
 'METRONIDAZOLE 400mg',
 'CLOTRIMAZOLE Powder',
 'FLUCANAZOLE 150mg',
 'AZITHROMYCIN 500mg',
 'ITRACONAZOLE 100 mg',
 'VITAMIN E 400 mg',
 'Povidone-Iodine IP 5%',
 'AZITHROMYCIN 250mg',
 'AMLODIPINE BISILATE 10mg',
 'VITAMIN D3 60K IU',
 'TRANEXAMIC ACID 500mg + MEFEMANIC ACID 250mg',
 'CIPROFLOXACIN 250mg',
 'Levosalbutamol Sulphate Ambroxol Hydrochloride and Guaiphenesin Syrup',
 'NORFLOXACIN 400mg',
 'MEFENAMIC ACID 500mg',
 'OFLOXACIN Eye/Ear Drops 5ml',
 'CETIRIZINE syrup',
 'Paracetamol, phenylephrine hydrochloride and chlorpheniramine maleate suspension (Paediatric)',
 'MEFENAMIC ACID 250mg + DICYCLOMINE HCL 10mg',
 'CEFIXIME ORAL SUPENSION 50mg/30ml',
 'ORS',
 'DOXYCYCLINE HCL 100 mg',
 'VITAMIN B Complex +C (Sy

In [0]:
ratios['HEMATINIC SYRUP - Protein']


0

In [0]:
print(y[4:7].to_string())

4    [AMOXICILLIN 500mg + POTASSIUM CLAVULANATE 5G/...
5    [PCM ORAL SUSPENSION 250mg, PCM ORAL SUSPENSIO...
6    [PANTOPRAZOLE 40 mg, PARACETAMOL 500mg, DICLOF...


In [0]:
[ratio for ratio in ratios.keys() if ratios[ratio] < 1] 

['HEMATINIC SYRUP - Protein']

In [0]:
ratios, mean_ir

(defaultdict(int,
             {'ACECLOFENAC 100mg': 4.307349665924276,
              'ACECLOFENAC 100mg + PARACETAMOL 325mg': 3.4290780141843973,
              'ALBENDAZOLE 400mg': 12.517799352750808,
              'AMBROXOL HCl 15mg': 107.44444444444444,
              'AMINOPHYLLINE 100mg': 257.8666666666667,
              'AMLODIPINE BISILATE 10mg': 39.876288659793815,
              'AMLODIPINE BISILATE 2.5mg': 9.209523809523809,
              'AMLODIPINE BISILATE 5mg': 3.173092698933552,
              'AMOXICILLIN 125mg': 96.7,
              'AMOXICILLIN 250mg': 17.662100456621005,
              'AMOXICILLIN 500mg': 9.550617283950617,
              'AMOXICILLIN 500mg + POTASSIUM CLAVULANATE 5G/30ML': 36.149532710280376,
              'AMOXICILLIN 500mg + POTASSIUM CLAVULANIC ACID 125mg': 12.318471337579618,
              'AMOXICILLIN ORAL SUPENSION 125mg/30ml': 77.36,
              'AMPICILLIN 250mg': 322.3333333333333,
              'AMPICILLIN 250mg + CLOXACILLIN 250mg': 80.58333

In [0]:
sc, sc_ratios = SCUMBLE(dataset[1])

AMOXICILLIN ORAL SUPENSION 125mg/30ml 50
PCM ORAL SUSPENSION 250mg 24
RANITIDINE 150 mg 1710
AMOXICILLIN 500mg 405
AMOXICILLIN 250mg 419
AMOXICILLIN 500mg + POTASSIUM CLAVULANATE 5G/30ML 107
CEFIXIME 100mg 76
AMLODIPINE BISILATE 5mg 1219
PCM ORAL SUSPENSION 125mg 30
PCM ORAL SUSPENSION 120mg 18
ATENOLOL 50mg 370
PANTOPRAZOLE 40 mg 3240
PARACETAMOL 500mg 3191
DICLOFENAC Ointment 2998
VITAMIN B Complex +C 931
VITAMIN B Complex 1957
ATENOLOL 25mg 332
METFORMIN 500mg 1194
LOSARTAN 50mg 237
METFORMIN 500mg - SR 114
GLIMEPERIDE 2 mg + METFORMIN 500mg 462
FERROUS ASCORBATE 100mg + FOLIC ACID 1.5mg 594
METRONIDAZOLE 400mg 337
CLOTRIMAZOLE Powder 10
FLUCANAZOLE 150mg 337
CETIRIZINE Di HCl 10mg 1606
AZITHROMYCIN 500mg 434
ACECLOFENAC 100mg 898
CALCIUM CARBONATE 1250mg & VIT D3 250 IU 3868
ITRACONAZOLE 100 mg 65
CEFIXIME 200mg 449
VITAMIN E 400 mg 106
HEMATINIC SYRUP - Protein, Iron, Folic Acid and Vitamin B 12 (pediatric) 292
DERIPHYLLINE 100 mg 263
Povidone-Iodine IP 5% 32
AMOXICILLIN 500mg + P

In [0]:
len([sc_ratio for sc_ratio in sc_ratios if sc_ratio > sc])

8252

In [0]:
sc

0.12443896964456251

In [0]:
mean_ir, ratios = MeanIR(dataset[1], True)


AMOXICILLIN ORAL SUPENSION 125mg/30ml 50
PCM ORAL SUSPENSION 250mg 24
RANITIDINE 150 mg 1710
AMOXICILLIN 500mg 405
AMOXICILLIN 250mg 419
AMOXICILLIN 500mg + POTASSIUM CLAVULANATE 5G/30ML 107
CEFIXIME 100mg 76
AMLODIPINE BISILATE 5mg 1219
PCM ORAL SUSPENSION 125mg 30
PCM ORAL SUSPENSION 120mg 18
ATENOLOL 50mg 370
PANTOPRAZOLE 40 mg 3240
PARACETAMOL 500mg 3191
DICLOFENAC Ointment 2998
VITAMIN B Complex +C 931
VITAMIN B Complex 1957
ATENOLOL 25mg 332
METFORMIN 500mg 1194
LOSARTAN 50mg 237
METFORMIN 500mg - SR 114
GLIMEPERIDE 2 mg + METFORMIN 500mg 462
FERROUS ASCORBATE 100mg + FOLIC ACID 1.5mg 594
METRONIDAZOLE 400mg 337
CLOTRIMAZOLE Powder 10
FLUCANAZOLE 150mg 337
CETIRIZINE Di HCl 10mg 1606
AZITHROMYCIN 500mg 434
ACECLOFENAC 100mg 898
CALCIUM CARBONATE 1250mg & VIT D3 250 IU 3868
ITRACONAZOLE 100 mg 65
CEFIXIME 200mg 449
VITAMIN E 400 mg 106
HEMATINIC SYRUP - Protein, Iron, Folic Acid and Vitamin B 12 (pediatric) 292
DERIPHYLLINE 100 mg 263
Povidone-Iodine IP 5% 32
AMOXICILLIN 500mg + P

In [0]:
mean_ir

334.83362306745295

In [0]:
data_divided = pd.DataFrame({'symptoms':dataset[0], 'label':dataset[1]})

In [0]:
data_divided.shape


(20704, 2)

In [0]:
data_divided.to_hdf('../UC_datasets/pure_datasets/REMEDIAL/data.hdf','df',format='fixed')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['symptoms', 'label']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [0]:
!ls ../UC_datasets/pure_datasets/REMEDIAL

data.hdf
